In [ ]:
!pip install transformers
!git clone https://github.com/aub-mind/arabert
!pip install torch
import transformers
from transformers import AutoTokenizer, AutoModel
import torch
import os
import nltk
import pickle
nltk.download('stopwords')
from nltk.corpus import stopwords
from collections import Counter
import string

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 23.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 34.9 MB/s eta 0:00:00
Cloning into 'arabert'...
remote: Enumerating objects: 600, done.
remote: Counting objects: 100% (65/65), done.
remote: Compressing objects: 100% (33/33), done.
remote: Total 600 (delta 38), reused 45 (delta 30), pack-reused 535
Receiving objects: 100% (600/600), 9.14 MiB | 25.51 MiB/s, done.
Resolving deltas: 100% (339/339), done.


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
# define the corpus
corpus1 = ".تعتبر قناة السويس ممر مائي يربط بين البحر المتوسط والبحر الأحمر منذ القرون القديمة"

corpus2 = ".قناة يوتيوب تعتبر رقمية لأنها تسمح للمستخدمين بمشاهدة و مشاركة مقاطع الفيديو عبر الإنترنت"

In [ ]:
# remove stopwords
stop_words = set(stopwords.words("arabic"))
# get frequency
# Create a frequency distribution for corpus1
freq_dist1 = nltk.FreqDist()

# Update the frequency distribution with the words in the sentence
freq_dist1.update(word for word in corpus1.split() if word not in stop_words)

# Print the frequency distribution
for word, freq in freq_dist1.items():
    print(word, freq)

# Save the frequency distribution for corpus_text1
freq_dist1_file = "corpus1_freq_dist.pkl"
with open(freq_dist1_file, "wb") as file:
    pickle.dump(freq_dist1, file)


.تعتبر 1
قناة 1
السويس 1
ممر 1
مائي 1
يربط 1
البحر 1
المتوسط 1
والبحر 1
الأحمر 1
القرون 1
القديمة 1


In [ ]:
# Create a frequency distribution for corpus2
freq_dist2 = nltk.FreqDist()

# Update the frequency distribution with the words in the sentence
freq_dist2.update(word for word in corpus2.split() if word not in stop_words)

# Print the frequency distribution
for word, freq in freq_dist2.items():
    print(word, freq)

# Save the frequency distribution for corpus_text1
freq_dist2_file = "corpus2_freq_dist.pkl"
with open(freq_dist2_file, "wb") as file:
    pickle.dump(freq_dist2, file)

.قناة 1
يوتيوب 1
تعتبر 1
رقمية 1
لأنها 1
تسمح 1
للمستخدمين 1
بمشاهدة 1
مشاركة 1
مقاطع 1
الفيديو 1
عبر 1
الإنترنت 1


In [ ]:
# Load the frequency distributions from pickle files
with open("corpus1_freq_dist.pkl", "rb") as file:
    freq_dist1 = pickle.load(file)

with open("corpus2_freq_dist.pkl", "rb") as file:
    freq_dist2 = pickle.load(file)

# Remove punctuation marks
translator = str.maketrans("", "", string.punctuation)
freq_dist1_cleaned = {word.translate(translator) for word in freq_dist1}
freq_dist2_cleaned = {word.translate(translator) for word in freq_dist2}

# Get the common words between the two frequency distributions
target_words = freq_dist1_cleaned.intersection(freq_dist2_cleaned)
# Save the target_words as a pickle file
with open("target_words.pkl", "wb") as file:
    pickle.dump(target_words, file)
# Print the target words
print("Target words:", target_words)

Target words: {'تعتبر', 'قناة'}


In [ ]:
import pandas as pd
# Remove punctuation marks
translator = str.maketrans("", "", string.punctuation)
freq_dist1_cleaned = {word.translate(translator): freq for word, freq in freq_dist1.items()}
freq_dist2_cleaned = {word.translate(translator): freq for word, freq in freq_dist2.items()}

target_words = set(freq_dist1_cleaned).intersection(set(freq_dist2_cleaned))

# Create a DataFrame for target words and their frequencies
data = {"Target Word": [], "Corpus1 Frequency": [], "Corpus2 Frequency": []}

for word in target_words:
    data["Target Word"].append(word)
    data["Corpus1 Frequency"].append(freq_dist1_cleaned[word])
    data["Corpus2 Frequency"].append(freq_dist2_cleaned[word])

df = pd.DataFrame(data)
# Drop the index column
df = df.reset_index(drop=True)
# Print the DataFrame
print(df.reset_index(drop=True))

  Target Word  Corpus1 Frequency  Corpus2 Frequency
0       تعتبر                  1                  1
1        قناة                  1                  1


In [ ]:
# loading the model + tokenizer
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_name = "aubmindlab/bert-base-arabertv2" # can change to CAMeLBERT CA and MSA too!

# uncomment the model you would like to see the scores for
# model_name = "CAMeL-Lab/bert-base-arabic-camelbert-msa"
# model_name = "CAMeL-Lab/bert-base-arabic-camelbert-ca"

model = AutoModel.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = model.to(device)

In [ ]:
# tokenize corpus1
tokenized_corpus1 = tokenizer.encode_plus(corpus1, return_tensors='pt')
# Access the input tensors
input_ids1 = tokenized_corpus1['input_ids']
attention_masks1 = tokenized_corpus1['attention_mask']
token_type_ids1 = tokenized_corpus1['token_type_ids']
print(tokenized_corpus1['input_ids'][0])
print(tokenizer.convert_ids_to_tokens(tokenized_corpus1['input_ids'][0]))

tensor([   33,    48,  2277,  1852,   251, 18696,   220,  4758,  4941,  7296,
          407, 27825,   211, 30218, 13173,  6689, 16779,  5665,  4449,   211,
          731, 18316,   313, 19259,   871,   251,    34])
['[CLS]', '.', 'تعتبر', 'قنا', '##ة', 'السوي', '##س', 'ممر', 'مائي', 'يربط', 'بين', 'البح', '##ر', 'المت', '##وسط', 'وال', '##بحر', 'الأ', '##حم', '##ر', 'منذ', 'القر', '##ون', 'الق', '##ديم', '##ة', '[SEP]']


In [ ]:
# tokenize corpus2
tokenized_corpus2 = tokenizer.encode_plus(corpus2, return_tensors = 'pt')
# Access the input tensors
input_ids2 = tokenized_corpus2['input_ids']
attention_masks2 = tokenized_corpus2['attention_mask']
token_type_ids2 = tokenized_corpus2['token_type_ids']
print(tokenized_corpus2['input_ids'][0])
print(tokenizer.convert_ids_to_tokens(tokenized_corpus2['input_ids'][0]))

tensor([   33,    48,  1852,   251, 11127,  2277,  6134,   251,  4849,  2331,
         4600,   162,  7306,   210, 34848,   219, 46849,  1161,   251,   166,
          783,   251,  3285, 32800,  3318,   982,  4404,  1376, 25906,   210,
           34])
['[CLS]', '.', 'قنا', '##ة', 'يوتيوب', 'تعتبر', 'رقمي', '##ة', 'لأن', '##ها', 'تسمح', 'ل', '##لمس', '##ت', '##خدمي', '##ن', 'بمش', '##اهد', '##ة', 'و', 'مشارك', '##ة', 'مقاطع', 'الفي', '##ديو', 'عبر', 'الإ', '##نت', '##رن', '##ت', '[SEP]']


In [ ]:
model.eval()

# Choose the last layer
selected_layers = [11]  # Last layer

# Tokenize the sentence from corpus1
encoded_dict_sentence1 = tokenizer.encode_plus(
    corpus1,
    add_special_tokens=True,
    max_length=256,
    padding='max_length',
    return_attention_mask=True,
    return_tensors='pt'
)

# Move input tensors to GPU
encoded_dict_sentence1 = {key: value.to('cuda') for key, value in encoded_dict_sentence1.items()}

# Get embeddings from BERT model for the sentence
with torch.no_grad():
    model_outputs_sentence1 = model(**encoded_dict_sentence1, output_hidden_states=True)
    all_hidden_states_sentence1 = model_outputs_sentence1.hidden_states

# Access the last hidden layer
selected_layer_tensors = [all_hidden_states_sentence1[-1]]

# Process each target word within the sentence
target_word_embeddings_batched1 = []

for target_word in target_words:
    # Tokenize the target word and get token IDs
    target_word_tokens = tokenizer.tokenize(target_word)
    target_word_token_ids = tokenizer.convert_tokens_to_ids(target_word_tokens)

    # Find the indices in the sentence tokens that correspond to the target word tokens
    target_indices_sentence = [i for i, token_id in enumerate(encoded_dict_sentence1['input_ids'][0]) if token_id in target_word_token_ids]

    # Get selected layers' embeddings for the sentence
    selected_layer_embeddings_sentence = torch.cat([layer[:, target_indices_sentence, :] for layer in selected_layer_tensors], dim=-1)

    # Calculate the mean embedding for the target word
    mean_target_embedding = torch.mean(selected_layer_embeddings_sentence, dim=1, keepdim=True)

    target_word_embeddings_batched1.append(mean_target_embedding)

# Convert the list of embeddings to a tensor
target_word_embeddings_batched1 = torch.cat(target_word_embeddings_batched1, dim=0)

# Print target words and their embeddings
for target_word, embedding in zip(target_words, target_word_embeddings_batched1):
    print("Target word:", target_word)
    print("Embedding:", embedding)
    print()

Target word: تعتبر
Embedding: tensor([[ 2.6876e-01,  7.0745e-02, -1.5843e-01,  6.5409e-02, -4.1756e-01,
          3.3427e-01,  4.0083e-01,  3.0574e-01,  3.2781e-02,  1.5818e-01,
         -2.0913e-01, -1.1370e+00, -2.2809e-01, -3.5428e-01,  6.0697e-01,
          6.2123e-01,  3.0154e-01,  2.6755e-01,  2.8171e-01, -3.7154e-01,
         -2.6869e-01,  1.5476e-01,  5.1940e-01,  1.9522e-01, -1.7501e-01,
         -7.8885e-01, -2.1007e-01, -4.8772e-01,  1.2627e+00, -3.4896e-01,
         -7.2731e-03, -4.6157e-01, -2.1273e-02, -4.8256e-01, -4.0989e-01,
          9.1009e-02, -3.6877e-01,  1.7513e-01, -5.0672e-02, -4.7508e-01,
         -7.9392e-02, -2.0642e-01,  1.3574e-01,  1.0147e-01,  7.8867e-02,
          3.3346e-01,  6.4079e-02,  2.2383e-01, -6.4135e-02,  5.7977e-01,
         -7.8346e-01,  4.1981e-01,  8.5256e-02,  2.6481e-01, -6.4162e-02,
          1.9584e-01,  8.7112e-01, -5.0907e-01, -3.4642e-01,  1.7461e-01,
         -4.1065e-01,  7.4707e-02, -7.3415e-02, -1.0819e-01,  2.6919e-01,
        

In [ ]:
model.eval()

# Choose the last layer
selected_layers = [11]  # Last layer

# Tokenize the sentence from corpus2
encoded_dict_sentence2 = tokenizer.encode_plus(
    corpus2,
    add_special_tokens=True,
    max_length=256,
    padding='max_length',
    return_attention_mask=True,
    return_tensors='pt'
)

# Move input tensors to GPU
encoded_dict_sentence2 = {key: value.to('cuda') for key, value in encoded_dict_sentence2.items()}

# Get embeddings from BERT model for the sentence
with torch.no_grad():
    model_outputs_sentence2 = model(**encoded_dict_sentence2, output_hidden_states=True)
    all_hidden_states_sentence2 = model_outputs_sentence2.hidden_states

# Access the last hidden layer
selected_layer_tensors = [all_hidden_states_sentence2[-1]]

# Process each target word within the sentence
target_word_embeddings_batched2 = []

for target_word in target_words:
    # Tokenize the target word and get token IDs
    target_word_tokens = tokenizer.tokenize(target_word)
    target_word_token_ids = tokenizer.convert_tokens_to_ids(target_word_tokens)

    # Find the indices in the sentence tokens that correspond to the target word tokens
    target_indices_sentence = [i for i, token_id in enumerate(encoded_dict_sentence2['input_ids'][0]) if token_id in target_word_token_ids]

    # Get selected layers' embeddings for the sentence
    selected_layer_embeddings_sentence = torch.cat([layer[:, target_indices_sentence, :] for layer in selected_layer_tensors], dim=-1)

    # Calculate the mean embedding for the target word
    mean_target_embedding = torch.mean(selected_layer_embeddings_sentence, dim=1, keepdim=True)

    target_word_embeddings_batched2.append(mean_target_embedding)

# Convert the list of embeddings to a tensor
target_word_embeddings_batched2 = torch.cat(target_word_embeddings_batched2, dim=0)

# Print target words and their embeddings
for target_word, embedding in zip(target_words, target_word_embeddings_batched2):
    print("Target word:", target_word)
    print("Embedding:", embedding)
    print()

Target word: تعتبر
Embedding: tensor([[ 8.1269e-02,  7.9171e-02, -1.0398e-01,  1.6586e-03, -1.6124e+00,
          9.1332e-01,  3.1328e-01,  6.5400e-01, -2.8379e-01,  2.0129e-01,
         -1.3734e-01, -1.2152e+00, -4.5432e-01, -3.2925e-01,  2.1770e-01,
          4.7691e-01, -2.8770e-02,  3.9312e-01,  2.2192e-01, -7.4738e-01,
         -1.6708e-01,  2.2875e-01,  8.3419e-01, -6.1682e-01, -4.1862e-01,
         -3.0231e-01, -8.0491e-01, -4.1752e-01,  9.1653e-01, -1.2115e-01,
          1.9610e-01, -3.2071e-01, -1.9181e-01, -1.0612e+00, -4.9784e-01,
          1.6007e-01, -9.4594e-01,  8.3069e-02, -1.5754e-01, -6.8005e-01,
         -1.4698e-01, -3.1763e-02,  1.0190e-01,  3.4343e-01,  1.9612e-01,
          2.5748e-01,  6.6084e-01, -1.3947e-01,  1.2307e-01,  3.5294e-01,
         -5.2303e-01,  2.1265e-02, -1.0067e-01,  3.2829e-01, -4.7251e-02,
          8.7037e-02,  5.3228e-02,  6.4575e-02, -2.1415e-02,  2.2931e-01,
         -3.1039e-01,  5.4268e-02, -1.8572e-01,  2.2324e-01,  2.6352e-01,
        

In [ ]:
# Calculate cosine similarity scores for each target word pair
cosine_similarity_scores = []

for emb1, emb2 in zip(target_word_embeddings_batched1, target_word_embeddings_batched2):
    similarity_score = torch.cosine_similarity(emb1.unsqueeze(0), emb2.unsqueeze(0))
    cosine_similarity_scores.append(similarity_score.mean().item())  # using mean() to get scalar value

# Print cosine similarity scores for each target word
for target_word, similarity_score in zip(target_words, cosine_similarity_scores):
    print("Target word:", target_word)
    print("Cosine Similarity Score:", similarity_score)
    print()

Target word: تعتبر
Cosine Similarity Score: 0.4505208432674408

Target word: قناة
Cosine Similarity Score: 0.3932291865348816

